In [4]:
import PyPDF2
from langchain.docstore.document import Document
def extract_text(file_path, file_name):
    try:
        reader = PyPDF2.PdfReader(file_path)
        docs=[]
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text() if page.extract_text() else ""
            doc =  Document(page_content=text, metadata={"filename": file_name, "page_no":int(page_num)+1})
            docs.append(doc)
        return docs
    except Exception as e:
        print(f"Error reading PDF file: {e}")
        raise Exception("Error in PDF reading")

In [5]:
docs=extract_text("Journey_Documentation_ADSS.pdf","Journey_Documentation_ADSS")

In [6]:
docs

[Document(metadata={'filename': 'Journey_Documentation_ADSS', 'page_no': 1}, page_content=' \n \n \nJourney Documentation  \nOf \nADSS Trader Chatbot  \n27th Sep 2024  \n \n \n \n \n \n \nDOCUMENT INFORMATION  \n  \n  Information  \nDocument Owner  Abhishek V S  \nIssue Date  27-Sep -2024   \nLast Saved Date  27-Sept -2024   \nFile Name  Journey documentation of ADSS  \n '),
 Document(metadata={'filename': 'Journey_Documentation_ADSS', 'page_no': 2}, page_content=' \n1. Introduction  \nOverview of the ADSS Project  \nThe ADSS project focuses on developing an advanced ADSS  Trader chatbot designed to \nenhance user interactions in the trading domain. This chatbot leverages Natural Language \nUnderstanding (NLU) capabilities through Azure Cognitive Services, enabling it to \nproficiently analyze user queries in both English and Ara bic. The system is built to interpret \nvoice prompts, ensuring a versatile communication platform for users.  \nKey components of the project include:  \n•  

In [8]:
from openai import AzureOpenAI
from langchain_openai import OpenAIEmbeddings, AzureOpenAIEmbeddings
api_key="501e4271616c4a2baadda9bf1a669731"
azure_endpoint="https://openai-ct.openai.azure.com/"
api_version="2023-07-01-preview"
embedding_model_name="text-embedding-ada-002"

embeddings = AzureOpenAIEmbeddings(
            model=embedding_model_name, 
            api_key=api_key, 
            azure_endpoint=azure_endpoint, 
            openai_api_version=api_version)

In [17]:

from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
import os
import time
pc = Pinecone(api_key="pcsk_4hp8P_FGu8ZMCRX7gEjF1AH7ZTF3fv6V4uFyrTpNHkCidkvCofdF6LjR4QLwSZCpqSTGC")
os.environ['PINECONE_API_KEY'] = 'pcsk_4hp8P_FGu8ZMCRX7gEjF1AH7ZTF3fv6V4uFyrTpNHkCidkvCofdF6LjR4QLwSZCpqSTGC'
index_name="test"

In [23]:

def vector_store() -> None:
    existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
    if index_name not in existing_indexes:
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1"),
        )
        while not pc.describe_index(index_name).status["ready"]:
            time.sleep(1)
    return PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [24]:
docsearch=vector_store()

In [28]:
docsearch.add_documents([docs[3]])

['59eaccda-dd05-4a1a-8f65-f6db14e7a861']

In [37]:
query="what are Initial Requirements of project?"
filters={
  "filename": { "$in": "Journey_Documentation_ADSS" }
}
docs=docsearch.similarity_search(query, k=1)

In [51]:
docs[0].page_content

"quality, and security. This documentation serves as a valuable resource for stakeholders to \nunderstand the journey of the project, from conception through execution, and highlights \nthe ongoing commitment to excellence in delivering a high -quality trading a ssistant.  \n 2. Client Expectations  \n Initial Requirements  \nThe initial requirements outlined by the client, as documented in the agreement facilitated \nby our sales team, included:  \n• Fast Response Time: The client emphasized the need for quick responses from the \nchatbot to enhance user experience and engagement.  \n• Natural Language Understanding : The ability to accurately comprehend and \nrespond to inquiries in both English and Arabic was a critical requirement.  \n• Seamless Integration:  The chatbot needed to integrate effectively with ADSS back -\nend systems and third -party APIs to provide up -to-date trading information.  \n• Error Handling : A robust error management system was required to ensure smooth \